In [1]:
import pandas as pd
import numpy as np
import sys
from pathlib import Path

sys.path.append(str(Path.cwd().parent))

from scripts.load import load_raw
from scripts.utils import run_paired_suite, summarize_against_threshold, motor_stat_row_joint
from scripts.config import DATA_PROCESSED_PATH, VEL_MIN, VEL_MAX, VEL_NORMAL_UMBRAL, \
RESULTS_PATH, RESULTS_TABLES, BINARY_COLS

In [2]:
# Parámetros/constantes
CSV_NAME = "data-neuro-tts.csv"
UMBRAL_VEL = VEL_NORMAL_UMBRAL

In [3]:
# Carga de datos
df = load_raw(CSV_NAME)
print(f'El dataset tiene {df.shape[0]} filas y {df.shape[1]} columnas.')

El dataset tiene 49 filas y 57 columnas.


In [4]:
df.head(3) # Primeros 3 registros

,id,tts_pie_derecho,tts_pie_izquierdo,radiculopatia_s1_dch,radiculopatia_s1_izq,polineuropatia,arcada_del_soleo,pre_fecha,pre_dch_motor_velocidad_total,pre_dch_motor_amplitud_total,...,post_dch_sensitivo_velocidad_n_calcaneo_medial,post_dch_sensitivo_amplitud_n_calcaneo_medial,post_izq_sensitivo_velocidad_n_plantar_medial,post_izq_sensitivo_amplitud_n_plantar_medial,post_izq_sensitivo_velocidad_n_plantar_lateral,post_izq_sensitivo_amplitud_n_plantar_lateral,post_izq_sensitivo_velocidad_n_baxter,post_izq_sensitivo_amplitud_n_baxter,post_izq_sensitivo_velocidad_n_calcaneo_medial,post_izq_sensitivo_amplitud_n_calcaneo_medial
0,Y3870090D,1,0,0,0,0,0,2022-12-21,52.2,23.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,50297827R,1,1,1,1,0,0,2023-03-18,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,50735751M,1,0,1,0,0,0,2023-01-23,48.2,8.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df.tail(3) # Últimos 3 registros

,id,tts_pie_derecho,tts_pie_izquierdo,radiculopatia_s1_dch,radiculopatia_s1_izq,polineuropatia,arcada_del_soleo,pre_fecha,pre_dch_motor_velocidad_total,pre_dch_motor_amplitud_total,...,post_dch_sensitivo_velocidad_n_calcaneo_medial,post_dch_sensitivo_amplitud_n_calcaneo_medial,post_izq_sensitivo_velocidad_n_plantar_medial,post_izq_sensitivo_amplitud_n_plantar_medial,post_izq_sensitivo_velocidad_n_plantar_lateral,post_izq_sensitivo_amplitud_n_plantar_lateral,post_izq_sensitivo_velocidad_n_baxter,post_izq_sensitivo_amplitud_n_baxter,post_izq_sensitivo_velocidad_n_calcaneo_medial,post_izq_sensitivo_amplitud_n_calcaneo_medial
46,15374842D,1,0,0,0,0,0,2025-07-03,53.3,7.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,01831288M,1,1,0,1,0,0,2024-06-26,52.5,15.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,75914456N,0,1,0,0,0,1,2024-06-05,NaN,NaN,...,NaN,NaN,50.6,1.5,50.0,0.3,NaN,NaN,NaN,NaN


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 57 columns):
 #   Column                                          Non-Null Count  Dtype         
---  ------                                          --------------  -----         
 0   id                                              49 non-null     object        
 1   tts_pie_derecho                                 49 non-null     Int64         
 2   tts_pie_izquierdo                               49 non-null     Int64         
 3   radiculopatia_s1_dch                            49 non-null     Int64         
 4   radiculopatia_s1_izq                            49 non-null     Int64         
 5   polineuropatia                                  49 non-null     Int64         
 6   arcada_del_soleo                                49 non-null     Int64         
 7   pre_fecha                                       49 non-null     datetime64[ns]
 8   pre_dch_motor_velocidad_total                   38 n

## Duplicados

In [7]:
# Verificar si existen duplicados
dup_mask = df["id"].duplicated(keep=False)
n_ids_duplicated = df.loc[dup_mask, "id"].nunique()
n_rows_dup_id = dup_mask.sum()

print(f"IDs duplicados (n IDs): {n_ids_duplicated}")
print(f"Filas con id duplicado (n filas): {n_rows_dup_id}")

IDs duplicados (n IDs): 0
Filas con id duplicado (n filas): 0


## Valores faltantes

In [8]:
missing_pct = df.isna().mean().sort_values(ascending=False) * 100
missing_tbl = missing_pct.to_frame("pct_missing").round(2)
display(missing_tbl.head(40))

,pct_missing
post_dch_sensitivo_amplitud_n_calcaneo_medial,100.00
post_dch_sensitivo_velocidad_n_calcaneo_medial,100.00
post_izq_sensitivo_amplitud_n_calcaneo_medial,97.96
post_dch_sensitivo_velocidad_n_baxter,97.96
post_dch_sensitivo_amplitud_n_baxter,97.96
post_izq_sensitivo_velocidad_n_baxter,97.96
post_izq_sensitivo_amplitud_n_baxter,97.96
post_izq_sensitivo_velocidad_n_calcaneo_medial,97.96
pre_dch_sensitivo_velocidad_n_calcaneo_medial,95.92
post_dch_sensitivo_velocidad_n_plantar_medial,95.92


In [9]:
# Guardar para trazabilidad
RESULTS_TABLES.mkdir(parents=True, exist_ok=True)
missing_tbl.to_csv(RESULTS_TABLES / "porcentajes_valores_faltantes.csv")

## Análisis descriptivo general

In [10]:
n_estudios = df[["tts_pie_derecho","tts_pie_izquierdo"]].agg(["sum", "count"])
n_pacientes = df['id'].nunique()

In [11]:
n_estudios

,tts_pie_derecho,tts_pie_izquierdo
sum,42,32
count,49,49


In [12]:
print(f"Total de pacientes: {n_pacientes}")

Total de pacientes: 49


### Conteos de pies (diagnóstico por lado)

In [13]:
# Disponibles (no NA)
total_dch_disp = df["tts_pie_derecho"].notna().sum()
total_izq_disp = df["tts_pie_izquierdo"].notna().sum()
total_pies_disp = int(total_dch_disp + total_izq_disp)

# Positivos (1) — skipna=True por si hay NA
total_dch_pos = int(df["tts_pie_derecho"].sum(skipna=True))
total_izq_pos = int(df["tts_pie_izquierdo"].sum(skipna=True))
total_pies_pos = int(total_dch_pos + total_izq_pos)

# Negativos (0) — skipna=True por si hay NA
total_dch_neg = int((df["tts_pie_derecho"] == 0).sum(skipna=True))
total_izq_neg = int((df["tts_pie_izquierdo"] == 0).sum(skipna=True))
total_pies_neg = int(total_dch_neg + total_izq_neg)

print(f"Total pies disponibles (no NA): {total_pies_disp} ({total_dch_disp} dcho + {total_izq_disp} izq)")
print(f"Total pies TTS positivos (1): {total_pies_pos} ({total_pies_pos/total_pies_disp:.1%}) ({total_dch_pos} dcho + {total_izq_pos} izq)")
print(f"Total pies TTS negativos (0): {total_pies_neg} ({total_pies_neg/total_pies_disp:.1%}) ({total_dch_neg} dcho + {total_izq_neg} izq)")

Total pies disponibles (no NA): 98 (49 dcho + 49 izq)
Total pies TTS positivos (1): 74 (75.5%) (42 dcho + 32 izq)
Total pies TTS negativos (0): 24 (24.5%) (7 dcho + 17 izq)


In [14]:
# columnas motor velocidad total/segmentario por lado y tiempo
COL_PRE_DCH_VEL_TOT = "pre_dch_motor_velocidad_total"
COL_POST_DCH_VEL_TOT = "post_dch_motor_velocidad_total"
COL_PRE_IZQ_VEL_TOT = "pre_izq_motor_velocidad_total"
COL_POST_IZQ_VEL_TOT = "post_izq_motor_velocidad_total"

COL_PRE_DCH_VEL_SEG = "pre_dch_motor_velocidad_segmentario"
COL_POST_DCH_VEL_SEG = "post_dch_motor_velocidad_segmentario"
COL_PRE_IZQ_VEL_SEG = "pre_izq_motor_velocidad_segmentario"
COL_POST_IZQ_VEL_SEG = "post_izq_motor_velocidad_segmentario"

cols_desc = [
    COL_PRE_DCH_VEL_TOT, COL_POST_DCH_VEL_TOT, 
    COL_PRE_IZQ_VEL_TOT, COL_POST_IZQ_VEL_TOT,
    COL_PRE_DCH_VEL_SEG, COL_POST_DCH_VEL_SEG, 
    COL_PRE_IZQ_VEL_SEG, COL_POST_IZQ_VEL_SEG
]

In [15]:
desc_motor_vel = df[cols_desc].describe().T  # Transponer para tener métricas en filas
desc_motor_vel = desc_motor_vel.round(2)     # Redondear los valores numéricos a 2 decimales
desc_motor_vel


,count,mean,std,min,25%,50%,75%,max
pre_dch_motor_velocidad_total,38.0,50.03,4.43,42.2,46.80,48.90,53.10,61.2
post_dch_motor_velocidad_total,12.0,51.35,4.22,45.9,48.18,50.00,54.50,58.0
pre_izq_motor_velocidad_total,32.0,49.28,3.73,43.0,46.45,49.15,51.62,58.3
post_izq_motor_velocidad_total,11.0,49.65,6.96,35.7,47.00,49.30,51.70,64.5
pre_dch_motor_velocidad_segmentario,42.0,39.35,5.14,29.7,36.40,39.05,41.62,53.6
post_dch_motor_velocidad_segmentario,22.0,44.91,4.03,36.8,43.47,45.20,46.95,52.2
pre_izq_motor_velocidad_segmentario,35.0,39.72,5.31,27.1,36.55,39.40,43.50,50.7
post_izq_motor_velocidad_segmentario,18.0,48.51,4.78,39.3,45.72,47.70,52.48,57.1


In [16]:
# Resetear índice para tener 'métrica' como columna
desc_motor_vel = desc_motor_vel.reset_index().rename(columns={"index": "metrica"})

# Guardar análisis descriptivo
desc_motor_vel.to_csv(RESULTS_TABLES / "analisis_descriptivo_motor_velocidad.csv", index=False)

In [17]:
# Añadir columnas con el total (izq + dcho) de pre y post, total y segmentario
groups = [
    (["pre_dch_motor_velocidad_total",  "pre_izq_motor_velocidad_total"],  "motor_vel_total_pre"),
    (["post_dch_motor_velocidad_total", "post_izq_motor_velocidad_total"], "motor_vel_total_post"),
    (["pre_dch_motor_velocidad_segmentario",  "pre_izq_motor_velocidad_segmentario"],  "motor_vel_seg_pre"),
    (["post_dch_motor_velocidad_segmentario", "post_izq_motor_velocidad_segmentario"], "motor_vel_seg_post"),
]

# Construir tabla resumen
rows = [motor_stat_row_joint(df, cols, label) for cols, label in groups]
tbl_motor_stats = pd.DataFrame(rows, columns=["Variable","n","media","sd","min","max"])
tbl_motor_stats = tbl_motor_stats.round(2)
tbl_motor_stats

,Variable,n,media,sd,min,max
0,motor_vel_total_pre,70,49.69,4.11,42.20,61.20
1,motor_vel_total_post,23,50.53,5.63,35.70,64.50
2,motor_vel_seg_pre,77,39.52,5.19,27.10,53.60
3,motor_vel_seg_post,40,46.53,4.69,36.80,57.10


In [18]:
# Tiempos entre estudios pre y post
df["dias_entre_estudios"] = (df["post_fecha"] - df["pre_fecha"]).dt.days
display(df["dias_entre_estudios"].describe())

count     27.000000
mean     339.407407
std      117.024409
min       60.000000
25%      310.500000
50%      328.000000
75%      349.000000
max      741.000000
Name: dias_entre_estudios, dtype: float64

In [19]:
df["semanas_entre_estudios"] = [(col / 7) for col in df["dias_entre_estudios"]]
display(df["semanas_entre_estudios"].describe())

count     27.000000
mean      48.486772
std       16.717773
min        8.571429
25%       44.357143
50%       46.857143
75%       49.857143
max      105.857143
Name: semanas_entre_estudios, dtype: float64

## Conteo de positivos por velocidad

Dentro de los pies con TTS=1 (derecho o izquierdo):

1. **Velocidad total (motor_velocidad_total):**
   - Positivos al test (patológicos): < 45
   - Negativos al test (normales): ≥ 45

2. **Velocidad segmentaria (motor_velocidad_segmentario):**
   - Positivos: < 45
   - Negativos: ≥ 45

Recuento de pies en cada categoría y el % sobre el total de positivos.

In [20]:
# Máscaras de pies TTS+
mask_dch_pos = df["tts_pie_derecho"] == 1
mask_izq_pos = df["tts_pie_izquierdo"] == 1

# Series PRE por pies TTS+ (cada pie = una observación)
# Concatenar derecho e izquierdo para construir "todos los TTS+".
s_total_pre = pd.concat([
    df.loc[mask_dch_pos, "pre_dch_motor_velocidad_total"],
    df.loc[mask_izq_pos, "pre_izq_motor_velocidad_total"]
], ignore_index=True)

s_seg_pre = pd.concat([
    df.loc[mask_dch_pos, "pre_dch_motor_velocidad_segmentario"],
    df.loc[mask_izq_pos, "pre_izq_motor_velocidad_segmentario"]
], ignore_index=True)

# Denominador = nº de pies TTS+ (dinámico; equivale a suma de 1s)
total_pies_pos = int(mask_dch_pos.sum() + mask_izq_pos.sum())
print(f"Total de pies TTS+: {total_pies_pos}") # Verificación

Total de pies TTS+: 74


In [21]:
# Resumen para total y segmentaria (PRE)
row_total = summarize_against_threshold(s_total_pre, total_pies_pos, UMBRAL_VEL)
row_total.name = "velocidad_total_pre"

row_seg = summarize_against_threshold(s_seg_pre, total_pies_pos, UMBRAL_VEL)
row_seg.name = "velocidad_segmentaria_pre"

res_tts_pos_pre = pd.DataFrame([row_total, row_seg])
res_tts_pos_pre.head().T

,velocidad_total_pre,velocidad_segmentaria_pre
total_tts_pos,74.00,74.00
n_validos,67.00,73.00
n_faltantes,7.00,1.00
n_positivos_<umbral,5.00,67.00
n_negativos_≥umbral,62.00,6.00
%_pos_sobre_tts_pos,6.76,90.54
%_neg_sobre_tts_pos,83.78,8.11
%_faltantes_sobre_tts_pos,9.46,1.35


In [22]:
# Metadatos y guardado
res_tts_pos_pre.insert(0, "umbral_test_positivo", f"< {UMBRAL_VEL}")
RESULTS_TABLES.mkdir(parents=True, exist_ok=True)
out_file = RESULTS_TABLES / "tts_positivos_umbral_pre.csv"
res_tts_pos_pre.to_csv(out_file)
print(f"Guardado: /results/tables/tts_positivos_umbral_pre.csv")
# print(f"Guardado: {out_file}")

Guardado: /results/tables/tts_positivos_umbral_pre.csv


## Análisis velocidad motora vs sensitiva

- Tomamos solo pies con TTS positivo.
- Dentro de esos, filtramos los pies cuya velocidad motora total es negativa (normal), es decir ≥ 45 en PRE.
- Sobre ese subconjunto (por lado), contamos cuántos son positivos sensoriales (< 45) en cada nervio: n_plantar_medial, n_plantar_lateral, n_baxter.
- Obtenemos los conteos por lado y el total (derecho+izquierdo).

In [23]:
# Motor total NEGATIVO (normal) en PRE por lado (>= umbral)
mask_motorneg_dch = df["pre_dch_motor_velocidad_total"] >= UMBRAL_VEL
mask_motorneg_izq = df["pre_izq_motor_velocidad_total"] >= UMBRAL_VEL

# Subconjuntos elegibles: TTS+ y motor total NEGATIVO (normal)
eligible_dch = df[mask_dch_pos & mask_motorneg_dch]
eligible_izq = df[mask_izq_pos & mask_motorneg_izq]

# Denominadores (cuántos pies entran en esta evaluación por lado)
denom_dch = len(eligible_dch)
denom_izq = len(eligible_izq)

print(f"Total pies TTS+ y motor total normal (>= {UMBRAL_VEL}) en PRE:")
print(f"- Total: {denom_dch + denom_izq} ({(denom_dch + denom_izq) / total_pies_pos * 100:.2f}% del total de positivos)")
print(f"    - Derecho: {denom_dch} ({denom_dch / (denom_dch + denom_izq) * 100:.2f}%)")
print(f"    - Izquierdo: {denom_izq} ({denom_izq / (denom_dch + denom_izq) * 100:.2f}%)")


Total pies TTS+ y motor total normal (>= 45.0) en PRE:
- Total: 62 (83.78% del total de positivos)
    - Derecho: 35 (56.45%)
    - Izquierdo: 27 (43.55%)


In [24]:
# Columnas sensitivas (PRE) por nervio y lado
sens_cols = {
    "n_plantar_medial": {
        "dch": "pre_dch_sensitivo_velocidad_n_plantar_medial",
        "izq": "pre_izq_sensitivo_velocidad_n_plantar_medial",
    },
    "n_plantar_lateral": {
        "dch": "pre_dch_sensitivo_velocidad_n_plantar_lateral",
        "izq": "pre_izq_sensitivo_velocidad_n_plantar_lateral",
    },
    "n_baxter": {
        "dch": "pre_dch_sensitivo_velocidad_n_baxter",
        "izq": "pre_izq_sensitivo_velocidad_n_baxter",
    },
}

# Contar “positivos sensoriales” (< umbral) entre los elegibles
rows = []
for nerve, sides in sens_cols.items():
    # Derecho
    s_d = eligible_dch[sides["dch"]] if denom_dch > 0 else pd.Series(dtype=float)
    npos_d = int((s_d < VEL_NORMAL_UMBRAL).sum(skipna=True))
    nval_d = int(s_d.notna().sum()) if denom_dch > 0 else 0

    # Izquierdo
    s_i = eligible_izq[sides["izq"]] if denom_izq > 0 else pd.Series(dtype=float)
    npos_i = int((s_i < VEL_NORMAL_UMBRAL).sum(skipna=True))
    nval_i = int(s_i.notna().sum()) if denom_izq > 0 else 0

    # Totales
    npos_tot = npos_d + npos_i
    denom_tot = denom_dch + denom_izq
    nval_tot = nval_d + nval_i

    rows.append({
        "nervio": nerve,
        "n_eligibles_total": denom_tot,
        "n_sens_pos": npos_tot,
        "%_sens_pos_sobre_eligibles": round((npos_tot / denom_tot * 100), 2) if denom_tot else np.nan,

        # info de datos válidos (por si hay NA en velocidad sensorial)
        "n_valid_total": nval_tot,
        "n_valid_dch": nval_d,
        "n_valid_izq": nval_i,
    })

h3_table = pd.DataFrame(rows)

In [25]:
nerves_order = ["n_plantar_medial", "n_plantar_lateral", "n_baxter"]
rows_order = [
    "n_eligibles_total",
    "n_sens_pos",
    "%_sens_pos_sobre_eligibles",
    "n_valid_total",
    "n_valid_dch",
    "n_valid_izq",
]

# Dejar 'nervio' como índice y ordenar las columnas (nervios)
wide = (
    h3_table
    .set_index("nervio")
    .loc[nerves_order]        # ordenar nervios (columnas futuras)
    .T                        # transpón: métricas -> filas, nervios -> columnas
)

# Reordenar filas
wide = wide.loc[rows_order]

# Reset índice para tener 'métrica' como columna
wide = wide.reset_index().rename(columns={"index": "metrica"})

# Insertar una fila inicial con los nombres de los nervios
header_row = pd.DataFrame([wide.columns], index=["nervio"], columns=wide.columns)
wide_with_header = pd.concat([header_row, wide])

# Quitar el primer registro (el índice 'nervio' de la fila insertada)
h3_table = wide_with_header.iloc[1:]

# Mostrar el resultado final
h3_table

nervio,metrica,n_plantar_medial,n_plantar_lateral,n_baxter
0,n_eligibles_total,62.0,62.0,62.0
1,n_sens_pos,22.0,21.0,7.0
2,%_sens_pos_sobre_eligibles,35.48,33.87,11.29
3,n_valid_total,27.0,24.0,7.0
4,n_valid_dch,15.0,13.0,3.0
5,n_valid_izq,12.0,11.0,4.0


- `n_eligibles_total`: número de pies TTS+ cuya velocidad motora total PRE ≥ 45 (es decir, TTS clínico/diagnóstico positivo pero motora total normal).
- `n_sens_pos`: número de pies dentro de los elegibles que son positivos sensoriales (< 45) en el nervio.
- `%_sens_pos_sobre_eligibles`: porcentaje de pies positivos sensoriales sobre los elegibles.
- `n_valid_total`: número de pies con datos válidos (no NA) en velocidad sensorial para ese nervio.
- `n_valid_dch`: número de pies con datos válidos (no NA) en velocidad sensorial para el nervio derecho.
- `n_valid_izq`: número de pies con datos válidos (no NA) en velocidad sensorial para el nervio izquierdo.

In [26]:
# Guardar en CSV
RESULTS_TABLES.mkdir(parents=True, exist_ok=True)
out_path = RESULTS_TABLES / "motora_neg_vs_sensitiva_pos_pre.csv"
h3_table.to_csv(out_path, index=False)
print(f"Guardado: /results/tables/motora_neg_vs_sensitiva_pos_pre.csv")
# print(f"Guardado: {out_path}")

Guardado: /results/tables/motora_neg_vs_sensitiva_pos_pre.csv


## Diferencia velocidad pre-post (total y segmentaria)

`Δ = post − pre`. De esta forma, Δ > 0 indica mejoría (más velocidad tras cirugía).

**Objetivo:** demostrar estadísticamente si, en promedio, hay **mejoría tras la cirugía** en cada métrica (velocidad total y segmentaria, pie dcho/izq).

Esto exige:

- **Contrastar hipótesis:** ¿la media del cambio (Δ = post − pre) es distinta de 0?
- **Cuantificar el tamaño del cambio:** ¿cuán grande es la mejoría? (tamaño del efecto, no solo p-valor).

Como son medidas del mismo pie antes y después (mismas unidades en dos momentos), los datos son pareados (muestras relacionadas). Usaremos **tests pareados**.

> **Importante:** “significativo” ≠ “clínicamente relevante”.

### t pareada

- Contrasta si la **media** de `Δ` es **0**.
- Asunción: los deltas (post − pre) siguen **aprox. normalidad**.
- Devuelve estadístico t y p-valor. Si p < α (p.ej., 0.05), rechazar H0 (media Δ = 0): hay cambio medio significativo

### Wilcoxon

- Alternativa **no paramétrica**; no asume normalidad.
- Contrasta si la **mediana** de `Δ` es 0 (técnicamente, simetría de la distribución de Δ respecto a 0).
- Útil si Δ es muy asimétrico o con **outliers**.
- Devuelve estadístico W y p-valor. Si p < α, rechazar H0 (mediana Δ = 0): hay cambio mediano significativo

### Tamaños del efecto: Cohen's d

Acompañan (no sustituyen) al p-valor y cuentan cuán grande es el cambio.

In [27]:
summary = run_paired_suite(df)
summary

,var,n_pairs,mean_delta,sd_delta,ci95_low,ci95_high,pct_improved,t_stat,p_t,p_t_fdr,w_stat,p_w,p_w_fdr,cohen_dz
0,vel_total_dch,11,0.7000,5.9947,-3.3273,4.7273,63.6364,0.3873,0.7067,0.9004,27.0,0.6377,0.8503,0.1487
1,vel_total_izq,11,-0.2545,6.5733,-4.6706,4.1615,45.4545,-0.1284,0.9004,0.9004,31.0,0.8984,0.8984,-0.0449
2,vel_seg_dch,20,5.5550,5.4301,3.0136,8.0964,80.0000,4.5750,0.0002,0.0004,18.0,0.0005,0.0010,1.1879
3,vel_seg_izq,18,8.6167,5.4489,5.9070,11.3264,100.0000,6.7091,0.0000,0.0000,0.0,0.0000,0.0000,1.8705


#### Interpretación de `summary`
- `mean_delta`, `ci95_low`, `ci95_high` → magnitud y precisión de la mejoría.
- `p_t_fdr` / `p_w_fdr` → significación tras corrección por múltiples comparaciones (elige la familia de tests principal según normalidad de Δ).
- `cohen_dz` → tamaño del efecto (≈0.2 pequeño, ≈0.5 medio, ≈0.8 grande; guía orientativa, poner en contexto clínico).

#### Resultados
- `vel_total_dcho`: La velocidad total motora del pie derecho muestra una mejora media pequeña y muy imprecisa (IC cruza 0). Los p-valores (t y Wilcoxon) no son significativos tras FDR. No podemos afirmar mejora media en “total”.
- `vel_total_izq`: Pie izquierdo: cambio medio nulo/ligeramente negativo, no significativo; IC amplio cruzando 0.
- `vel_seg_dcho`: Velocidad segmentaria derecha: mejora clara y grande, estadísticamente significativa tras FDR. Tamaño del efecto grande (≈1.2).
- `vel_seg_izq`: Segmentaria izquierda: mejora muy marcada, todos mejoran (100%), p<<0.05 tras FDR, efecto muy grande (≈1.9).

#### Conclusión rápida para tu estudio

- **H1 (mejora pre→post):** La conducción segmentaria mejora de forma consistente, grande y significativa en ambos pies (dcha e izda). La conducción total no muestra cambios significativos con esta muestra (n=11 por lado).
- **Implicación clínica:** La evaluación segmentaria parece más sensible para captar la mejoría post-cirugía que la medida total; encaja con nuestro H2 sobre “falsos negativos” cuando la total está normal y la segmentaria patológica.

> Nota: con n modesto en “total” (11 pares) la potencia es limitada; si más adelante crece la muestra, puede cambiar la precisión/decisión.

In [28]:
def format_row(r):
    sig = "sí" if ((pd.notna(r["p_t_fdr"]) and r["p_t_fdr"] < 0.05) or
                   (pd.notna(r["p_w_fdr"]) and r["p_w_fdr"] < 0.05)) else "no"
    ic = f'{r["ci95_low"]:.2f} – {r["ci95_high"]:.2f}'
    return {
        "Variable": r["var"],
        "n pares": int(r["n_pairs"]),
        "Δ̄ (m/s)": f'{r["mean_delta"]:.2f}',
        "IC95% (m/s)": ic,
        "% mejora": f'{r["pct_improved"]:.1f}%',
        "p_t (FDR)": f'{r["p_t_fdr"]:.4f}' if pd.notna(r["p_t_fdr"]) else "",
        "p_w (FDR)": f'{r["p_w_fdr"]:.4f}' if pd.notna(r["p_w_fdr"]) else "",
        "dᶻ": f'{r["cohen_dz"]:.2f}' if pd.notna(r["cohen_dz"]) else "",
        "¿Cambio significativo?": sig
    }

table_df = pd.DataFrame([format_row(r) for _, r in summary.iterrows()])

# Guardar CSV
table_df.to_csv(RESULTS_TABLES / "paired_summary_prepost.csv", index=False)

## Posibles falsos negativos y conteos

`fn_*` = falso negativo (total normal ≥ 45 y segmentario patológico < 45).

In [29]:
df["fn_pre_dch"] = (df[COL_PRE_DCH_VEL_TOT] >= UMBRAL_VEL) & (df[COL_PRE_DCH_VEL_SEG] < UMBRAL_VEL)
df["fn_pre_izq"] = (df[COL_PRE_IZQ_VEL_TOT] >= UMBRAL_VEL) & (df[COL_PRE_IZQ_VEL_SEG] < UMBRAL_VEL)
df["fn_post_dch"] = (df[COL_POST_DCH_VEL_TOT] >= UMBRAL_VEL) & (df[COL_POST_DCH_VEL_SEG] < UMBRAL_VEL)
df["fn_post_izq"] = (df[COL_POST_IZQ_VEL_TOT] >= UMBRAL_VEL) & (df[COL_POST_IZQ_VEL_SEG] < UMBRAL_VEL)

fn_counts = pd.DataFrame({
    "lado": ["derecho", "izquierdo"],
    "n_fn_pre": [
        df["fn_pre_dch"].sum(skipna=True),
        df["fn_pre_izq"].sum(skipna=True),
    ],
    "n_fn_post": [
        df["fn_post_dch"].sum(skipna=True),
        df["fn_post_izq"].sum(skipna=True),
    ]
})
fn_counts["pct_fn_pre"] = (fn_counts["n_fn_pre"] / len(df) * 100).round(2)
fn_counts["pct_fn_post"] = (fn_counts["n_fn_post"] / len(df) * 100).round(2)
display(fn_counts)

,lado,n_fn_pre,n_fn_post,pct_fn_pre,pct_fn_post
0,derecho,31,5,63.27,10.20
1,izquierdo,26,2,53.06,4.08


In [30]:
# Guardar Falsos negativos en archivo CSV
fn_counts.to_csv(RESULTS_TABLES / "falsos_negativos_vel.csv", index=False)

## Rangos y reglas clínicas

Conteo de outliers por columna (velocidad) en base a velocidad mínima y máxima definidas.

> **Nota**: Ajustar o revisar rangos fisiológicos (cambiar `VEL_MIN`/`VEL_MAX` en config)

In [31]:
vel_cols = [c for c in df.columns if "_velocidad_" in c]  # motor y sensitivo
outlier_rows = []

for col in vel_cols:
    s = df[col]
    mask_low  = s.notna() & (s < VEL_MIN)
    mask_high = s.notna() & (s > VEL_MAX)
    outlier_rows.append({
        "col": col,
        "n_low": int(mask_low.sum()),
        "n_high": int(mask_high.sum()),
        "n_total_outliers": int(mask_low.sum() + mask_high.sum()),
        "pct_outliers": round((mask_low.sum() + mask_high.sum()) / s.notna().sum() * 100, 2) if s.notna().sum() > 0 else np.nan
    })

outliers_tbl = pd.DataFrame(outlier_rows).sort_values("n_total_outliers", ascending=False)
display(outliers_tbl.head(20))

,col,n_low,n_high,n_total_outliers,pct_outliers
0,pre_dch_motor_velocidad_total,0,0,0,0.0
1,pre_dch_motor_velocidad_segmentario,0,0,0,0.0
22,post_izq_sensitivo_velocidad_n_baxter,0,0,0,0.0
21,post_izq_sensitivo_velocidad_n_plantar_lateral,0,0,0,0.0
20,post_izq_sensitivo_velocidad_n_plantar_medial,0,0,0,0.0
19,post_dch_sensitivo_velocidad_n_calcaneo_medial,0,0,0,NaN
18,post_dch_sensitivo_velocidad_n_baxter,0,0,0,0.0
17,post_dch_sensitivo_velocidad_n_plantar_lateral,0,0,0,0.0
16,post_dch_sensitivo_velocidad_n_plantar_medial,0,0,0,0.0
15,post_izq_motor_velocidad_segmentario,0,0,0,0.0


In [32]:
# Guardar outliers en csv (si hay)
count_outliers = outliers_tbl["n_total_outliers"].sum()
if (count_outliers > 0):
    print(f"Hay {count_outliers} outliers")
    outliers_tbl.to_csv(config.RESULTS_TABLES / "outliers_velocidad.csv", index=False)
else:
    print(f"No hay outliers")

No hay outliers


## Consistencia binarios

In [33]:
# Verificar {0, 1, NA} en binarios. Mostrar valores distintos si existen
bad_values = {}

for col in BINARY_COLS:
    if col in df.columns:
        uniques = set(df[col].dropna().unique().tolist())
        invalid = [v for v in uniques if v not in (0,1)]
        if invalid:
            bad_values[col] = invalid

bad_values  # {} si todo OK

{}

## Guardar dataset procesado

In [34]:
# Guardar CSV
df.to_csv(DATA_PROCESSED_PATH / "data-neuro-tss.csv", index=False)
print("Guardado: /data/processed/data-neuro-tss.csv")
# print(f"Guardado: {DATA_PROCESSED_PATH}/data-neuro-tss.csv")

Guardado: /data/processed/data-neuro-tss.csv


## Guardar resumen QA

In [35]:
# Exportar diccionario breve en results/summary_qa.md con bullets de issues detectados
summary_lines = []

summary_lines.append(f"# Resumen QA — {CSV_NAME}\n")
summary_lines.append(f"- **Shape**: {df.shape[0]} filas × {df.shape[1]} columnas")
summary_lines.append(f"- **Pacientes únicos (id)**: {df['id'].nunique()}")

# Muestra total por pies (disponibles y positivos)
summary_lines.append(
    f"- **Muestra total (pies disponibles)**: {total_pies_disp} "
    f"(derechos: {total_dch_disp}, izquierdos: {total_izq_disp})"
)
summary_lines.append(
    f"- **Pies con TTS positivo**: {total_pies_pos} ({total_pies_pos/total_pies_disp:.1%}) "
    f"(derechos: {total_dch_pos} ({total_dch_pos/total_pies_pos:.1%}), izquierdos: {total_izq_pos} ({total_izq_pos/total_pies_pos:.1%}))"
)
summary_lines.append(
    f"- **Pies con TTS negativo**: {total_pies_neg} ({total_pies_neg/total_pies_disp:.1%}) "
    f"(derechos: {total_dch_neg} ({total_dch_neg/total_pies_neg:.1%}), izquierdos: {total_izq_neg} ({total_izq_neg/total_pies_neg:.1%}))"
)

# Duplicados
summary_lines.append(f"- **IDs duplicados**: {n_ids_duplicated} (filas afectadas: {n_rows_dup_id})\n")

# Missing top-10
top_missing = missing_tbl.head(20).reset_index().rename(columns={"index":"col"})
summary_lines.append("## Top-20 columnas con más valores faltantes\n")
for _, r in top_missing.iterrows():
    summary_lines.append(f"- {r['col']}: {r['pct_missing']}%")

# Análisis descriptivo motor velocidad
summary_lines.append("\n## Análisis descriptivo de velocidad motora (por lado)\n")
summary_lines.append(tbl_motor_stats.to_markdown(index=False))

# Total pies TTS+ y motor total normal (>= {UMBRAL_VEL}) en PRE
summary_lines.append(f"\n## Total pies TTS+ y motor total normal (>= {UMBRAL_VEL}) en PRE\n")
summary_lines.append(f"- Total: {denom_dch + denom_izq} "
                     f"({(denom_dch + denom_izq) / total_pies_pos * 100:.2f}% del total de positivos)")
summary_lines.append(f"    - Derecho: {denom_dch} ({denom_dch / (denom_dch + denom_izq) * 100:.2f}%)")
summary_lines.append(f"    - Izquierdo: {denom_izq} ({denom_izq / (denom_dch + denom_izq) * 100:.2f}%)")

# Falsos negativos
summary_lines.append("\n## H2 — Posibles falsos negativos")
summary_lines.append("\n### Pre cirugía")
for _, r in fn_counts.iterrows():
    summary_lines.append(f"- {r['lado']}: n={int(r['n_fn_pre'])} ({r['pct_fn_pre']}%)")

summary_lines.append("\n### Post cirugía")
for _, r in fn_counts.iterrows():
    summary_lines.append(f"- {r['lado']}: n={int(r['n_fn_post'])} ({r['pct_fn_post']}%)")

# Outliers (velocidad)
summary_lines.append("\n## Outliers en velocidad (rango usado: "
                     f"{VEL_MIN}–{VEL_MAX} m/s; ajustable)\n")

if (count_outliers > 0):
    top_out = outliers_tbl.head(10).to_dict("records")
    for r in top_out:
        summary_lines.append(f"- {r['col']}: n_out={r['n_total_outliers']} "
                             f"(low={r['n_low']}, high={r['n_high']}, "
                             f"{r['pct_outliers']}% de válidos)")
else:
    summary_lines.append("- No se detectaron outliers.")

# Binarios
summary_lines.append("\n## Consistencia de binarios\n")
if bad_values:
    for col, vals in bad_values.items():
        summary_lines.append(f"- {col}: valores no válidos {vals}")
else:
    summary_lines.append("- OK (solo {0,1,NA})")

# Fechas
if "pre_fecha" in df.columns and "post_fecha" in df.columns:
    dias = (df["post_fecha"] - df["pre_fecha"]).dt.days
    n_neg = int((dias < 0).sum())
    summary_lines.append("\n## Fechas\n")
    summary_lines.append(f"- Días entre pre y post: n={dias.notna().sum()}, "
                         f"mediana={np.nanmedian(dias)}, IQR≈({np.nanpercentile(dias,25)}, {np.nanpercentile(dias,75)})")
    summary_lines.append(f"- Semanas entre pre y post: n={dias.notna().sum()}, "
                         f"mediana={round(np.nanmedian(dias / 7), 2)}, IQR≈({round(np.nanpercentile(dias / 7,25), 2)}, {round(np.nanpercentile(dias / 7,75), 2)})")
    summary_lines.append(f"- Casos con post < pre: {n_neg}")

# Guardar markdown
summary_md = "\n".join(summary_lines)
with open(RESULTS_PATH / "summary_qa.md", "w", encoding="utf-8") as f:
    f.write(summary_md)

print("Guardado: /results/summary_qa.md")
# print(f"Guardado: {RESULTS_PATH}/summary_qa.md")

Guardado: /results/summary_qa.md
